<a href="https://colab.research.google.com/github/Srikanth2719/MACHINE-LEARING-WITH-PYTHON-/blob/main/FAKE_NEWS_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data pre processing

In [ ]:
#loading the dataset to a pandas dataframe
news_dataset = pd.read_csv('/content/train.csv')

In [ ]:
news_dataset.shape

(20800, 5)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+'  '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus  House Dem Aide: We Didn’t Even ...
1        Daniel J. Flynn  FLYNN: Hillary Clinton, Big W...
2        Consortiumnews.com  Why the Truth Might Get Yo...
3        Jessica Purkiss  15 Civilians Killed In Single...
4        Howard Portnoy  Iranian woman jailed for ficti...
                               ...                        
20795    Jerome Hudson  Rapper T.I.: Trump a ’Poster Ch...
20796    Benjamin Hoffman  N.F.L. Playoffs: Schedule, M...
20797    Michael J. de la Merced and Rachel Abrams  Mac...
20798    Alex Ansary  NATO, Russia To Hold Parallel Exe...
20799             David Swanson  What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object
0        Darrell Lucus  House Dem Aide: We Didn’t Even ...
1        Daniel J. Flynn  FLYNN: Hillary Clinton, Big W...
2        Consortiumnews.com  Why the Truth Might Get Yo...
3        Jessica Purkiss  15 Civilians Killed In Single...
4        Howard Portnoy  Iranian woman jailed for ficti...
            

In [ ]:
# separating the data & label
X= news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [ ]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

stemming:

it is the process of reducing a word into Root word

example:
actor,actress,action-->act

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0              : ’ ’
1            . : , -
2                  .
3                 15
4                   
            ...     
20795       . .: ’ ’
20796    . . . : , -
20797        . ’ ’ -
20798              ,
20799            -35
Name: content, Length: 20800, dtype: object


In [ ]:
# separating the data & label
X= news_dataset['content'].values
Y= news_dataset['label'].values

In [ ]:
print(X)

[': ’ ’' '. : , -' '.' ... '. ’ ’ -' ',' '-35']


In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
Y.shape

(20800,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)



In [ ]:
print(X)

  (3, 62)	1.0
  (27, 62)	1.0
  (31, 145)	0.4249057758414498
  (31, 20)	0.49454392483696136
  (31, 7)	0.7582093299765108
  (36, 146)	0.7333171625424529
  (36, 131)	0.6798867105045412
  (37, 220)	0.8630627144132533
  (37, 145)	0.5050967738855863
  (40, 131)	1.0
  (47, 1203)	0.3691999620908306
  (47, 1123)	0.38440673900713157
  (47, 1057)	0.3691999620908306
  (47, 1039)	0.3691999620908306
  (47, 1013)	0.38440673900713157
  (47, 938)	0.38440673900713157
  (47, 868)	0.38440673900713157
  (53, 1385)	1.0
  (56, 251)	1.0
  (61, 47)	1.0
  (73, 348)	1.0
  (75, 263)	1.0
  (76, 263)	0.7876361228177006
  (76, 1)	0.6161406803910127
  (78, 243)	1.0
  :	:
  (20680, 1)	0.6161406803910127
  (20690, 1096)	0.3159830601044173
  (20690, 1088)	0.3159830601044173
  (20690, 1078)	0.3159830601044173
  (20690, 892)	0.3159830601044173
  (20690, 799)	0.2244087729413534
  (20690, 783)	0.2244087729413534
  (20690, 780)	0.3159830601044173
  (20690, 733)	0.3034830609713406
  (20690, 705)	0.2663660348465984
  (20690, 6

splitting the dataset to training & test data


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)

Training the Model: LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)


In [ ]:
print('Accuracy score of the training data :', training_data_accuracy)

Accuracy score of the training data : 0.5408653846153846


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)


In [ ]:
print('Accuracy score of the test data:',test_data_accuracy)

Accuracy score of the test data: 0.5228365384615384


Making a Predictive System

In [ ]:
X_new = X_test[0]

prediction =model.predict(X_new)
print(prediction)

if(prediction[0]==0):
   print('the news is Real')
else:
    print('the news is False')

[0]
the news is Real


In [ ]:
print(Y_test[3])

0
